In [1]:
import MeCab
from bs4 import BeautifulSoup
import requests
import logging
import jaconv
from gensim import corpora
from gensim import models
from gensim.models import word2vec
from janome.tokenizer import Tokenizer
from pprint import pprint
import pandas as pd, numpy as np

In [3]:
url = "https://www.aozora.gr.jp/access_ranking/2019_xhtml.html"
URL = ""
res = requests.get(url)
res.encoding = 'shift-jis'
soup = BeautifulSoup(res.content, "html.parser")

In [4]:
url_list = [url["href"] for i, url in enumerate(soup.find_all("a", target="_blank")) if i < 50]

In [5]:
title = []
name = []
text = []
for url in url_list:
    res = requests.get(url)
    url_start = url[:37]
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    for i, a in enumerate(soup.find_all("a")):
        if i == 7:
            url_end = a["href"][1:]
    url = url_start + url_end
    res = requests.get(url)
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    title.append(soup.find("h1").string)
    name.append(soup.find("h2").string)
    for tag in soup.find_all(["rt", "rp"]):
        tag.decompose()
    text.append(soup.find("div",{'class': 'main_text'}).get_text())

In [6]:
df = pd.DataFrame({'タイトル': title, '作者': name, '本文': text})

In [7]:
df.head()

,タイトル,作者,本文
0,〔雨ニモマケズ〕,宮澤賢治,\r\n雨ニモマケズ\r\n風ニモマケズ\r\n雪ニモ夏ノ暑サニモマケヌ\r\n丈夫ナカラダ...
1,走れメロス,太宰治,\r\n メロスは激怒した。必ず、かの邪智暴虐の王を除かなければならぬと決意した。メロスには...
2,山月記,中島敦,\r\n 隴西の李徴は博学才穎、天宝の末年、若くして名を虎榜に連ね、ついで江南尉に補せられた...
3,こころ,夏目漱石,\n上 先生と私\n\n\n一\n\r\n 私はその人を常に先生と呼んでいた。だからここでも...
4,羅生門,芥川龍之介,\r\n ある日の暮方の事である。一人の下人が、羅生門の下で雨やみを待っていた。\r\n 広...


In [3]:
mecab = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
print(mecab.parse("雨ニモマケズ風ニモマケズ"))
print(mecab.parse("雨にもまけず風にもまけず"))
print(mecab.parse("ポケモンGO"))

雨ニモマケズ	名詞,固有名詞,一般,*,*,*,雨ニモマケズ,アメニモマケズ,アメニモマケズ
風	名詞,接尾,一般,*,*,*,風,フウ,フー
ニモマケズ	名詞,一般,*,*,*,*,*
EOS

雨	名詞,一般,*,*,*,*,雨,アメ,アメ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
も	助詞,係助詞,*,*,*,*,も,モ,モ
まけ	動詞,自立,*,*,一段,未然形,まける,マケ,マケ
ず	助動詞,*,*,*,特殊・ヌ,連用ニ接続,ぬ,ズ,ズ
風	名詞,一般,*,*,*,*,風,カゼ,カゼ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
も	助詞,係助詞,*,*,*,*,も,モ,モ
まけ	動詞,自立,*,*,一段,未然形,まける,マケ,マケ
ず	助動詞,*,*,*,特殊・ヌ,連用ニ接続,ぬ,ズ,ズ
EOS

ポケモンGO	名詞,固有名詞,一般,*,*,*,ポケモンGO,ポケモンゴー,ポケモンゴー
EOS



In [11]:
!mecab -D

filename:	/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/sys.dic
version:	102
charset:	UTF8
type:	0
size:	4668394
left size:	1316
right size:	1316



In [6]:
mecab = MeCab.Tagger("/etc/mecabrc")
info = mecab.dictionary_info()
print(info.charset)

utf8


In [24]:
!cat /etc/mecabrc

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n

